In [1]:
import pandas as pd
import uuid
import random
import json

### Fix on Google Drive

In [12]:
# df = pd.read_pickle('../data/teaching.pkl')
# sermons = df[df.category=='sermon'].rename(columns={'title':'title_raw'})
# def get_title(s):
#     try:
#         return s[:s.rfind('(')].strip()
#     except:
#         return ''

# def get_passage(s):
#     try:
#         return s[s.rfind('('):].strip().replace('(','').replace(')','')
#     except:
#         return ''
                
# sermons['title'] = sermons.title_raw.apply(lambda x: get_title(x))
# sermons['passage'] = sermons.title_raw.apply(lambda x: get_passage(x))
# sermons = sermons.sort_values('date_str').reset_index(drop=True)
# sermons.to_excel('../data/temp_sermons_for_google_drive.xlsx')

### Prep for Sanity schema

In [12]:
def new_audio_link(audioLink):
    try:
        return audioLink.split('.org/')[-1].replace('/','___')
    except:
        return ""

In [18]:
sermons_edited = pd.read_excel('../data/sermons.xlsx')


In [5]:
sermons_edited = sermons_edited.drop(columns=['Unnamed: 0',])
sermons_edited['mp3File'] = sermons_edited.audioLink.apply(lambda x: new_audio_link(x))
sermons_edited = sermons_edited.drop(columns=['date'])
sermons_edited = sermons_edited.rename(columns={'date_str':'date', 'audioLink':'oldAudioLink'})
sermons_edited = sermons_edited.reset_index().rename(columns={'index':'_id', 'series':'sermonSeries'})
sermons_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(sermons_edited.shape[0])]
sermons_edited['_type'] = 'sermons'
sermons_edited = sermons_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sermonSeries', 'mp3File', 'oldAudioLink']]
sermons_edited = sermons_edited.fillna("")

In [6]:
sermons_edited = sermons_edited[sermons_edited.date <= "2022-10-31"]

sermons_edited['book'] = sermons_edited.book.apply(lambda x: 'Hebrews' if x == 'Hebrew' else x)

sermons_edited.drop(columns=['mp3File']).to_json("../cms/sermons.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)


In [7]:
sermons_edited.head(2)

,_id,_type,date,speaker,book,passage,title,sermonSeries,mp3File,oldAudioLink
0,79ad4898-8f88-11ed-8d0d-00000001732a,sermons,2005-08-07,Todd Wilson,1 Corinthians,11:23-34,Faithful Partakers,1 Corinthians,media___1_Corinthians___mp3___1_Corinthians_27...,https://www.gracecovenantbaptist.org/media/1_C...
1,79ad496a-8f88-11ed-bc4a-00000000b78d,sermons,2005-08-14,Todd Wilson,1 Corinthians,1:1-9,Paul and Corinth,1 Corinthians,media___1_Corinthians___mp3___1_Corinthians_1.mp3,https://www.gracecovenantbaptist.org/media/1_C...


In [ ]:
mp3List = sermons_edited[['_id','mp3File']].rename(columns={'_id':'assetId', 'mp3File':'mp3FileName'}).to_dict('records')
with open('../cms/mp3List.js', 'w') as f:
    f.write(f"export const mp3s = {json.dumps(mp3List)}")

### Sunday School for Sanity

In [20]:
df = pd.read_pickle('../data/teaching_clean.pkl')
ss = df[df.category=='sunday school'].rename(columns={'title':'title_raw'})
def get_title(s):
    try:
        return s[:s.rfind('(')].strip()
    except:
        return ''

def get_passage(s):
    try:
        return s[s.rfind('('):].strip().replace('(','').replace(')','')
    except:
        return ''
                
ss['title'] = ss.title_raw.apply(lambda x: get_title(x))
ss['passage'] = ss.title_raw.apply(lambda x: get_passage(x))
ss = ss.sort_values('date_str').reset_index(drop=True)
ss.to_excel('../data/temp_ss_for_google_drive.xlsx')

In [21]:
ss.head()

,date,speaker,title_raw,audioLink,handoutLink,testament,book,date_str,year,category,title,passage
0,09/04/2005,Michael Gaydosh,Daniel 1,https://www.gracecovenantbaptist.org/media/Dan...,NaN,OT,Daniel,2005-09-04,2005,sunday school,Daniel 1,
1,09/11/2005,Steve Cowan,Daniel 2,https://www.gracecovenantbaptist.org/media/Dan...,NaN,OT,Daniel,2005-09-11,2005,sunday school,Daniel 2,
2,09/18/2005,Michael Gaydosh,Daniel 3,https://www.gracecovenantbaptist.org/media/Dan...,NaN,OT,Daniel,2005-09-18,2005,sunday school,Daniel 3,
3,10/02/2005,Jayson Faulkner,Daniel 4,https://www.gracecovenantbaptist.org/media/Dan...,NaN,OT,Daniel,2005-10-02,2005,sunday school,Daniel 4,
4,10/09/2005,Steve Cowan,Daniel 5,https://www.gracecovenantbaptist.org/media/Dan...,NaN,OT,Daniel,2005-10-09,2005,sunday school,Daniel 5,


In [19]:
sermons_edited.head(1)

,Unnamed: 0,category,date,date_str,year,speaker,title_raw,audioLink,handoutLink,title,passage,book,testament,series
0,0,sermon,08/07/2005,2005-08-07,2005,Todd Wilson,Faithful Partakers (11:23-34),https://www.gracecovenantbaptist.org/media/1_C...,NaN,Faithful Partakers,11:23-34,1 Corinthians,NT,1 Corinthians


In [14]:
ss_edited = ss_edited.drop(columns=['Unnamed: 0',])
ss_edited['mp3File'] = ss_edited.audioLink.apply(lambda x: new_audio_link(x))
ss_edited = ss_edited.drop(columns=['date'])
ss_edited = ss_edited.rename(columns={'date_str':'date', 'audioLink':'oldAudioLink'})
ss_edited = ss_edited.reset_index().rename(columns={'index':'_id', 'series':'sermonSeries'})
ss_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(ss_edited.shape[0])]
ss_edited['_type'] = 'sermons'
ss_edited = ss_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sermonSeries', 'mp3File', 'oldAudioLink']]
ss_edited = ss_edited.fillna("")

KeyError: "['Unnamed: 0'] not found in axis"